<a href="https://colab.research.google.com/github/Amit198181/JBVNL/blob/main/MDI%3E%3D1_05_CD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  # Read the CSV into a pandas DataFrame
  try:
    df = pd.read_excel(fn)
    print('\nDataFrame head:')
    display(df.head())
  except Exception as e:
    print('Error reading file: {e}'.format(e=e))

Saving HIGH_VALUE_APR_AUG.xlsx to HIGH_VALUE_APR_AUG.xlsx
User uploaded file "HIGH_VALUE_APR_AUG.xlsx" with length 335580 bytes

DataFrame head:


,ZONE,CIRCLE,DIVISION,CONSNO,ACCOUNT_NO,CONSUMER_STATUS,CONSUMER_NAME,TARIFF_CODE,CONTRACTED_LOAD,CONSUMER_TYPE,...,LOAD_FACTOR_HTS_202506,SUPPLY_HOURS_HTS_202506,BILLED_UNITS_HTS_202507,MDI_HTS_202507,LOAD_FACTOR_HTS_202507,SUPPLY_HOURS_HTS_202507,BILLED_UNITS_HTS_202508,MDI_HTS_202508,LOAD_FACTOR_HTS_202508,SUPPLY_HOURS_HTS_202508
0,RANCHI,RANCHI,RANCHI EAST,NTMR11343,11220311098,LIVE,SUBODH SHARMA,CS Urban,60.0,RURAL,...,0.0,0.0,2113.0,12.00,0.0,0.0,2121.0,12.00,0.0,0.0
1,RANCHI,RANCHI,RANCHI WEST,RT24414,11220368914,LIVE,AYUSH FUELS,CS Urban,63.0,URBAN,...,0.0,0.0,4217.0,23.00,0.0,0.0,7124.0,25.00,0.0,0.0
2,RANCHI,RANCHI,RANCHI WEST,IT19014,11220410311,LIVE,SARYUG GAUTAM CONSTRUCTION PVT LTD PROP ANIL K...,CS Urban,40.0,URBAN,...,0.0,0.0,2451.0,18.98,0.0,0.0,1956.0,32.46,0.0,0.0
3,RANCHI,RANCHI,RANCHI WEST,B33178,11220430846,LIVE,SANJAY KUMAR TIWARI,CS Urban,32.0,URBAN,...,0.0,0.0,5088.0,20.20,0.0,0.0,5291.0,15.90,0.0,0.0
4,RANCHI,RANCHI,RANCHI WEST,IT21793,11220434115,LIVE,SATYA NARAYAN PRASAD CHOURASIYA,CS Urban,64.0,URBAN,...,0.0,0.0,3830.0,26.60,0.0,0.0,5492.0,25.94,0.0,0.0


In [5]:
# Identify MDI columns
mdi_columns = [col for col in df.columns if 'MDI_HTS_' in col]

# Convert 'CONTRACTED_LOAD' to numeric, coercing errors and filling NaNs
df['CONTRACTED_LOAD'] = pd.to_numeric(df['CONTRACTED_LOAD'], errors='coerce').fillna(0)

# Create a copy of the DataFrame to store exceedance information
df_exceedance = df.copy()

# Identify and mark exceedances
for mdi_col in mdi_columns:
    if mdi_col in df_exceedance.columns and 'CONTRACTED_LOAD' in df_exceedance.columns:
        # Calculate the ratio, avoid division by zero
        ratio = df_exceedance[mdi_col] / df_exceedance['CONTRACTED_LOAD'].replace(0, pd.NA)
        exceedance_mask = ratio >= 1.05
        # Create a new column to indicate exceedance for this month
        df_exceedance[f'EXCEEDED_{mdi_col}'] = exceedance_mask

# Filter rows where at least one exceedance occurred
consumers_with_exceedance = df_exceedance[df_exceedance.filter(like='EXCEEDED_').any(axis=1)].copy()

# Display the head of the consumers_with_exceedance DataFrame
print("Head of consumers_with_exceedance DataFrame:")
display(consumers_with_exceedance.head())

# Count the number of True values in 'EXCEEDED_' columns for each consumer
consumers_with_exceedance['EXCEEDANCE_COUNT'] = consumers_with_exceedance.filter(like='EXCEEDED_').sum(axis=1)

# Filter for consumers with exceedance count >= 4
consumers_with_4_or_more_exceedances = consumers_with_exceedance[consumers_with_exceedance['EXCEEDANCE_COUNT'] >= 4].copy()

# List of all columns to display
all_display_columns = ['ZONE', 'CIRCLE', 'DIVISION', 'CONSNO', 'ACCOUNT_NO', 'CONSUMER_STATUS', 'CONSUMER_NAME', 'TARIFF_CODE', 'CONTRACTED_LOAD', 'CONSUMER_TYPE', 'METER_READER_NAME', 'MF_KWH', 'PREPAID_FLAG', 'MTR_CONSUMP_TYPE', 'BILLED_UNITS_HTS_202504', 'MDI_HTS_202504', 'LOAD_FACTOR_HTS_202504', 'SUPPLY_HOURS_HTS_202504', 'BILLED_UNITS_HTS_202505', 'MDI_HTS_202505', 'LOAD_FACTOR_HTS_202505', 'SUPPLY_HOURS_HTS_202505', 'BILLED_UNITS_HTS_202506', 'MDI_HTS_202506', 'LOAD_FACTOR_HTS_202506', 'SUPPLY_HOURS_HTS_202506', 'BILLED_UNITS_HTS_202507', 'MDI_HTS_202507', 'LOAD_FACTOR_HTS_202507', 'SUPPLY_HOURS_HTS_202507', 'BILLED_UNITS_HTS_202508', 'MDI_HTS_202508', 'LOAD_FACTOR_HTS_202508', 'SUPPLY_HOURS_HTS_202508'] + [col for col in consumers_with_4_or_more_exceedances.columns if 'EXCEEDED_' in col] + ['EXCEEDANCE_COUNT']


# Display relevant columns, including the exceedance flags and the count
display(consumers_with_4_or_more_exceedances[all_display_columns])

Head of consumers_with_exceedance DataFrame:


,ZONE,CIRCLE,DIVISION,CONSNO,ACCOUNT_NO,CONSUMER_STATUS,CONSUMER_NAME,TARIFF_CODE,CONTRACTED_LOAD,CONSUMER_TYPE,...,SUPPLY_HOURS_HTS_202507,BILLED_UNITS_HTS_202508,MDI_HTS_202508,LOAD_FACTOR_HTS_202508,SUPPLY_HOURS_HTS_202508,EXCEEDED_MDI_HTS_202504,EXCEEDED_MDI_HTS_202505,EXCEEDED_MDI_HTS_202506,EXCEEDED_MDI_HTS_202507,EXCEEDED_MDI_HTS_202508
5,RANCHI,RANCHI,RANCHI WEST,CN13855,11220439252,LIVE,SANGITA DEVI,CS Urban,41.0,URBAN,...,0.0,8247.0,32.0,0.0,0.0,False,True,False,False,False
8,RANCHI,RANCHI,RANCHI WEST,RT27148,11220442429,LIVE,KARTIK GHOSH,CS Urban,45.0,URBAN,...,0.0,1520.0,64.0,0.0,0.0,True,False,True,False,True
36,RANCHI,RANCHI,RANCHI EAST,ANG16596,11220919563,LIVE,M/S VIVAN INDUSTRIES PVT.LTD. SRI VIVEK KUMAR,CS Urban,90.0,OTHERS,...,0.0,6390.0,88.0,0.0,0.0,False,False,True,True,False
38,RANCHI,RANCHI,RANCHI EAST,RP11991,11220921841,LIVE,SWAMI NAKULDEV,CS Urban,60.0,OTHERS,...,0.0,11526.0,58.4,0.0,0.0,False,False,False,True,False
61,RANCHI,RANCHI,RANCHI EAST,OM3D,11221072463,LIVE,DIRECTOR BIOLOGICAL PARK,CS Urban,40.0,OTHERS,...,0.0,6491.0,38.0,0.0,0.0,False,True,False,False,False


,ZONE,CIRCLE,DIVISION,CONSNO,ACCOUNT_NO,CONSUMER_STATUS,CONSUMER_NAME,TARIFF_CODE,CONTRACTED_LOAD,CONSUMER_TYPE,...,BILLED_UNITS_HTS_202508,MDI_HTS_202508,LOAD_FACTOR_HTS_202508,SUPPLY_HOURS_HTS_202508,EXCEEDED_MDI_HTS_202504,EXCEEDED_MDI_HTS_202505,EXCEEDED_MDI_HTS_202506,EXCEEDED_MDI_HTS_202507,EXCEEDED_MDI_HTS_202508,EXCEEDANCE_COUNT
87,RANCHI,RANCHI,RANCHI EAST,CHL23563,11221205010,LIVE,SAPNA SINGH,CS Urban,40.0,PRIVATE,...,16079.0,47.00,0.0,0.0,False,True,True,True,True,4
142,RANCHI,RANCHI,RANCHI KOKAR,KRK15506,11221566531,LIVE,CURESTA GLOBAL LIMITED,CS Urban,75.0,PRIVATE,...,28380.0,79.00,0.0,0.0,True,False,True,True,True,4
209,RANCHI,RANCHI,RANCHI KOKAR,BA4387,11221977764,LIVE,UPENDRA ROY,CS Urban,60.0,OTHERS,...,1612.0,58.00,0.0,0.0,True,True,True,True,False,4
221,RANCHI,RANCHI,RANCHI KOKAR,MOR4560,11222083673,LIVE,ANIRUDH BUDHIA,CS Urban,35.0,OTHERS,...,10820.0,42.00,0.0,0.0,True,True,True,True,True,5
254,RANCHI,RANCHI,RANCHI NEW CAPITAL,BU467,11222342554,LIVE,"SHARDA GLOBAL SCHOOL,",CS Urban,45.0,OTHERS,...,9466.0,80.00,0.0,0.0,True,True,True,True,True,5
336,RANCHI,RANCHI,RANCHI NEW CAPITAL,UB1700,11222735818,LIVE,BIMAL KUMAR MURARKA (CHEQ. NOT ACCEPT) S/O LT.,CS Urban,50.0,OTHERS,...,7412.0,54.00,0.0,0.0,True,True,True,True,True,5
396,RANCHI,RANCHI,RANCHI NEW CAPITAL,Ub5930,11222893911,LIVE,DEEPAK KINGER,CS Urban,40.0,PRIVATE,...,6554.0,37.00,0.0,0.0,True,True,True,True,False,4
406,RANCHI,RANCHI,RANCHI CENTRAL,N2471,11222969390,LIVE,RATAN KR.GUPTA,CS Urban,35.0,OTHERS,...,8630.0,55.00,0.0,0.0,True,True,True,True,True,5
485,RANCHI,RANCHI,RANCHI CENTRAL,T14174,11223171940,LIVE,HIL VIEW CONSTNICTION (P LTD.,CS Urban,75.0,OTHERS,...,2993.0,77.00,0.0,0.0,True,True,True,True,False,4
565,RANCHI,RANCHI,RANCHI CENTRAL,ANAG3231,11223747446,LIVE,THE SAILORS KNOT,CS Urban,40.0,PRIVATE,...,16298.0,49.00,0.0,0.0,True,True,False,True,True,4


In [6]:
# Export the consumers with 4 or more exceedances to an XLSX file
output_filename = 'consumers_with_4_or_more_exceedances.xlsx'
consumers_with_4_or_more_exceedances.to_excel(output_filename, index=False)

print(f'The sheet has been exported to {output_filename}. You can download it from the files pane.')

The sheet has been exported to consumers_with_4_or_more_exceedances.xlsx. You can download it from the files pane.


In [9]:
# Identify MDI columns for the last five months (assuming the last five in the list are the relevant ones)
mdi_columns_last_five = mdi_columns[-5:]

# Calculate the sum of MDI values for the months with exceedance and the count of such months for each consumer
consumers_with_4_or_more_exceedances['HIGH_DEMAND_MDI_SUM_LAST_FIVE'] = 0.0
consumers_with_4_or_more_exceedances['HIGH_DEMAND_MONTHS_COUNT_LAST_FIVE'] = 0

# Create a list to store the top 4 MDI values for each consumer
top_4_mdi_values = []

for index, row in consumers_with_4_or_more_exceedances.iterrows():
    mdi_values_in_exceedance_months = []
    for mdi_col in mdi_columns_last_five:
        exceedance_col = f'EXCEEDED_{mdi_col}'
        if exceedance_col in consumers_with_4_or_more_exceedances.columns and row[exceedance_col]:
            mdi_values_in_exceedance_months.append(row[mdi_col])

    # Sort MDI values in descending order and take the top 4
    mdi_values_in_exceedance_months.sort(reverse=True)
    top_4 = mdi_values_in_exceedance_months[:4]

    # Calculate the average of the top 4, handling cases with less than 4 exceedances
    average_top_4 = sum(top_4) / len(top_4) if len(top_4) > 0 else 0
    top_4_mdi_values.append(average_top_4)

# Assign the calculated average to a new column
consumers_with_4_or_more_exceedances['AVERAGE_TOP_4_MDI_HIGH_DEMAND_LAST_FIVE'] = top_4_mdi_values


# Display relevant columns including the new average
display(consumers_with_4_or_more_exceedances[['ZONE', 'CIRCLE', 'DIVISION', 'CONSNO', 'ACCOUNT_NO', 'CONSUMER_NAME', 'CONTRACTED_LOAD', 'AVERAGE_TOP_4_MDI_HIGH_DEMAND_LAST_FIVE', 'EXCEEDANCE_COUNT']])

,ZONE,CIRCLE,DIVISION,CONSNO,ACCOUNT_NO,CONSUMER_NAME,CONTRACTED_LOAD,AVERAGE_TOP_4_MDI_HIGH_DEMAND_LAST_FIVE,EXCEEDANCE_COUNT
87,RANCHI,RANCHI,RANCHI EAST,CHL23563,11221205010,SAPNA SINGH,40.0,55.2500,4
142,RANCHI,RANCHI,RANCHI KOKAR,KRK15506,11221566531,CURESTA GLOBAL LIMITED,75.0,83.2500,4
209,RANCHI,RANCHI,RANCHI KOKAR,BA4387,11221977764,UPENDRA ROY,60.0,84.0000,4
221,RANCHI,RANCHI,RANCHI KOKAR,MOR4560,11222083673,ANIRUDH BUDHIA,35.0,42.2500,5
254,RANCHI,RANCHI,RANCHI NEW CAPITAL,BU467,11222342554,"SHARDA GLOBAL SCHOOL,",45.0,74.5000,5
336,RANCHI,RANCHI,RANCHI NEW CAPITAL,UB1700,11222735818,BIMAL KUMAR MURARKA (CHEQ. NOT ACCEPT) S/O LT.,50.0,59.5000,5
396,RANCHI,RANCHI,RANCHI NEW CAPITAL,Ub5930,11222893911,DEEPAK KINGER,40.0,56.5000,4
406,RANCHI,RANCHI,RANCHI CENTRAL,N2471,11222969390,RATAN KR.GUPTA,35.0,58.7500,5
485,RANCHI,RANCHI,RANCHI CENTRAL,T14174,11223171940,HIL VIEW CONSTNICTION (P LTD.,75.0,82.7500,4
565,RANCHI,RANCHI,RANCHI CENTRAL,ANAG3231,11223747446,THE SAILORS KNOT,40.0,51.2500,4


In [11]:
# Export the current sheet to an XLSX file with the name "105"
output_filename = '105_persent.xlsx'
consumers_with_4_or_more_exceedances.to_excel(output_filename, index=False)

print(f'The sheet has been exported to {output_filename}. You can download it from the files pane.')

The sheet has been exported to 105_persent.xlsx. You can download it from the files pane.
